In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

x = np.linspace(0, 9, 10)
y = np.linspace(10, 19, 10)
lon, lat = np.meshgrid(x, y)
times = pd.date_range("2020-01-01", end = '2020-12-31', freq = '6D')
n = len(times)
data = np.random.randn(10,10, n)

ros = np.resize([1, 24], (n))
platforms = np.resize(['S1A', 'S1B', 'S1B', 'S1A'], (n))
direction = np.resize(['ascending', 'descending'], (n))

ds = xr.Dataset(data_vars= dict(
    data = (["x", "y", "time"], data)
        ),
    coords = dict(
                        lon = (["x", "y"], lon),
                        lat = (["x", "y"], lat),
                        time = times,
                        relative_orbit = (["time"], ros),
                        platform = (["time"], platforms),
                        flight_dir = (["time"], direction)) 
        
    )

import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/')
from spicy_snow.processing.s1_preprocessing import subset_s1_images, merge_s1_subsets

# import pickle
# with open('/Users/zachkeskinen/Documents/spicy-snow/data/main_test.pkl', 'rb') as f:
#     ds = pickle.load(f)

In [ ]:
dict_da = subset_s1_images(ds)

In [ ]:
ds = merge_s1_subsets(dict_da)

In [ ]:
ds